# Importa Pacotes

In [13]:
import os
import re
import locale
import numpy as np
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from gestao import ConectarAccess

# Definição de variaveis e pré-produção

In [ ]:
# Variáveis Definidas
dia_vencimento = 20
caminho_input = "C:/Users/vinir/Downloads/"
caminho_backup = "./Faturas Cartões/"

# Regex com o nome do arquivo, o primeiro grupo é o cartão e o segundo é mes da fatura
regex_nome_arquivo = "OUROCARD_([A-Z_]+)-(.*)\.txt"
regex_nome_arquivo_smile = "([A-Z_]+)-(.*)\.txt"
mask_arquivos = [
                 ('OUROCARD_ELO_GRAFITE.*', 'ELO GRAFITE') , 
                 ('OUROCARD_PLATINUM_VISA.*', 'PLATINUM VISA'),
                 ('OUROCARD_ELO_NANQUIM.*', 'ELO NANQUIM'),
                 ('SMILES_INFINITE_VISA.*','VISA INFINITE')
                ]

# Lista de ajuste do Tipo de Lancamento (Corrige o que vem da fatura)
ajuste_tipo = [('ASSAI','Supermercados'),('SUPER ADEGA','Supermercados')]

# Informações de base de dados
tabela_historico = '[historico_cartao]'
arquivo_base = 'GestaoFinanceira1'

#Variaveis de desenvolvimento
producao=True

print(f">> Execução em Produção: {producao}")
print(f">> Dia de Vencimento da Fatura: {dia_vencimento}")
print(f">> Pasta de Leitura dos arquivos de entrada: {caminho_input}")
print(f">> Arquivo com a Base em Access: {arquivo_base}")
print(f">> Tabela Com o Historico do Cartão: {tabela_historico}")
print(f">> Cartões: {', '.join([m[1] for m in mask_arquivos])}")
print(f">> Mascaras dos Arquivos: {', '.join([m[0] for m in mask_arquivos])}")

# Configuração do ambientes

In [ ]:
# Essa configuração é para o datetime entender os meses como mês Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

# Verificando arquivos presente na pasta

In [ ]:
files = []
for diretorio, subpastas, arquivos in os.walk(caminho_input):
    for arquivo in arquivos:
        for mask in mask_arquivos:
            if re.match(mask[0], arquivo):
                files.append(f"{diretorio}{arquivo}")
                print(f'[INFO] Localizado arquivo {arquivo}')

# Lendo os arquivos

In [ ]:
lista_lancamentos = []
total_fatura = []
for file in files:
    print(f'[INFO] Lendo o arquivo {file}')
    df = open(file, 'r') # Abre o arquivo de texto   
    # Essa condição é devido o cartão smile não ser Ourocard (tem nome diferente dos demais)
    if 'SMILES' in file:
        cartao = re.search(regex_nome_arquivo_smile, file).group(1).replace('_', ' ') 
    else:         
        cartao = re.search(regex_nome_arquivo, file).group(1).replace('_', ' ')  
    arquivo = file.replace(caminho_input,'')
    saldo_anterior = 0

    # Verifica se trata-se da próxima fatura
    if "Próxima_Fatura" in file: 
        prx_fat = True 
    else: 
        prx_fat = False
    prx_fat_parcelas = False

    l = df.readline()
    index = 0

    while l != '':
        index = index + 1

        # Captura saldo anterior para validação
        if 'SALDO FATURA ANTERIOR' in l:
            if prx_fat:
                saldo_anterior = float(l[59:74].strip().replace(".","").replace(",","."))
            else:
                saldo_anterior = float(l[59:72].strip().replace(".","").replace(",","."))

        # Condição de fim do arquivo
        if 'SubTotal' in l:
            # Lista para validação final
            total_fatura.append((arquivo,float(l[58:71].strip().replace(".","").replace(",",".")),saldo_anterior))
            break
    
        # Condição de quando existe compra internacinoal
        if '*** ' in l:
            print(f"[WARNING] Localizado compra internacional: {transacao}")
            print(f"[WARNING] O valor total da fatura pode divergir do total cobrado em conta corrente, devido o total da fatura não considera compras internacionais.")
            print(f"[WARNING] O DataFrame considera todas as compras, dessa forma condizente com o valor cobrando em conta corrente")
            l = df.readline()
            l = df.readline()
            l = df.readline()

        # Diversas condições para pegar apenas os lancamentos
        if l.strip() != '' and\
            ('.' in l[53:] or ',' in l[53:]) and\
            not('-----------' in tipo_lancamento) and\
            not('Saques' in l) and\
            not('SALDO FATURA ANTERIOR' in l) and\
            not('PGTO DEBITO CONTA' in l):
            #print(l)
            
            parcela = ''
            lancada = True

            if prx_fat:
                vencimento = datetime.now().date()
                data = l[0:5].strip() + "/" + ano
                valor = float(l[59:74].strip().replace(".","").replace(",","."))
                transacao = l[9:32].strip().replace("  ","")
            elif prx_fat_parcelas:
                vencimento = datetime.now().date()
                data = l[0:5].strip() + "/" + ano
                valor = float(l[59:74].strip().replace(".","").replace(",","."))
                transacao = l[10:24].strip().replace("  ","")
                pracela = l[29:34].strip()
            else:
                # Essa condição é devido o cartão smile não ser Ourocard (tem nome diferente dos demais)
                if 'SMILES' in file:
                    vencimento = re.search(regex_nome_arquivo_smile, file).group(2)
                else:
                    vencimento = re.search(regex_nome_arquivo, file).group(2)
                vencimento = datetime.strptime(f"{vencimento} {dia_vencimento}", '%b_%y %d').date()
                data = l[0:10].strip()
                valor = l[59:72].strip().replace(".","").replace(",",".")
                transacao = l[10:32].strip()

            if tipo_lancamento == 'Compras parceladas':
                if 'ANTEC ' in transacao:
                    transacao = l[10:24].strip()
                else:
                    transacao = l[10:23].strip()
                    parcela = l[29:34].strip()
                    

            if tipo_lancamento != '':
                lista_lancamentos.append(
                    [
                        datetime.strptime(data.replace(".","/"), '%d/%m/%Y').date(),
                        transacao, 
                        tipo_lancamento, 
                        float(valor), 
                        parcela, 
                        cartao, 
                        vencimento,
                        lancada,
                        arquivo,
                        index
                    ]
                )
        else:
            if not('PGTO DEBITO CONTA' in l): 
                tipo_lancamento = l.strip() #Define o tipo de lançamento
            if "Auto-Atendimento" in l: ano = l[6:10]
            # Quando começa a verifica as compras parcelas da próxima fatura
            if 'Compras/Pagamento de contas parceladas' in l and prx_fat: 
                prx_fat_parcelas = True


        l = df.readline()
    df.close()

    # Cria o dataFrame pandas e exporta para csv
    df_presente = pd.DataFrame(np.array(lista_lancamentos),
                                columns=[
                                            'Data_Compra',
                                            'Lancamento',
                                            'Tipo_Lancamento',
                                            'Valor',
                                            'Parcela',
                                            'Cartao',
                                            'Vencimento',
                                            'Lancada',
                                            'Arquivo',
                                            'Codigo'
                                        ]
                                ) 

    if producao:
        print(f"[INFO] Movendo arquivo {file} para pasta {caminho_backup}")
        if os.path.exists(f"{caminho_backup}{file.replace(caminho_input,'')}"):
            os.remove(f"{caminho_backup}{file.replace(caminho_input,'')}")
        os.rename(file, f"{caminho_backup}{file.replace(caminho_input,'')}")
        #os.remove(file)

# Ajustes dos dados
### Ajuste dos tipos de lançamentos

In [36]:
for ajs in ajuste_tipo:
    df_presente.loc[df_presente.Lancamento.str.contains(ajs[0]), 'Tipo_Lancamento']=ajs[1]

### Ajuste da data do vencimento da próxima fatura

In [ ]:
# Instancia o objeto que faz as conexões com a base de dados
acc = ConectarAccess(arquivo_base)

# Verifica a maior data presente na base que não é a próxima fatura
proximo_vencimento = acc.select("""
    SELECT MAX([Vencimento]) 
    FROM [historico_cartao] 
    WHERE NOT([Arquivo] LIKE '%Próxima%')
""")[0][0].date()

# Faz o máximo entre o maximo da base e o maximo do dataframe que não é a próxima fatura
max_df = df_presente[df_presente.Vencimento != datetime.now().date()].Vencimento.to_list()
if not(len(max_df) == 0):   
    proximo_vencimento = max(max(set(df_presente[df_presente.Vencimento != datetime.now().date()].Vencimento.to_list())), 
                         proximo_vencimento) + \
                    relativedelta(months=+1)
else:
    proximo_vencimento = proximo_vencimento + relativedelta(months=+1)

acc.close()

df_presente.loc[df_presente.Vencimento == datetime.now().date(), 'Vencimento'] = proximo_vencimento

print(f'Proximo Vencimento será {proximo_vencimento}')

# Insere informação na Base de Dados

In [ ]:
# Instancia o objeto que faz as conexões com a base de dados
acc = ConectarAccess(arquivo_base)

# Verifica se existe dados na base e caso exista exclui
# Isso é para permitir reprocessamento
for arq in df_presente.Arquivo.unique():
    verifica = acc.select(f"""
        SELECT COUNT(Arquivo)
        FROM {tabela_historico}
        WHERE Arquivo = '{arq}' 
        """)[0][0]
    if verifica:
        print(f"[INFO] Dados presente para Arquivo {arq}... Deletando dados da base")
        acc.delete(f"DELETE FROM {tabela_historico} WHERE Arquivo = '{arq}'")

acc.insertAcc(df_presente, tabela_historico)
print('[INFO] Dados Inseridos com sucesso')

# Fecha a conexão
acc.close()

# Validação

### Verificando Total Localizado na Fatura com o Total do DataFrame

In [ ]:
for tt in total_fatura:
    tota_df = df_presente[df_presente.Arquivo == tt[0]][['Arquivo','Valor']].groupby(['Arquivo']).sum().Valor.iloc[0]
    if abs(tota_df - tt[1]) < 0.0000001:
        print(f"[INFO] Total do Arquivo igual ao capturado no DataFrame para o Arquivo: {tt[0]}")
    else:
        if abs(abs(tt[2] - tt[1])-tota_df) < 0.0000001:
            print(f"[INFO] Total do Arquivo diverge do Dataframe devido não pagamento da fatura: {tt[0]}")
            #print(f"[INFO] Total do Arquivo igual ao capturado no DataFrame para o Arquivo: {tt[0]}")
        else:
            print(f"[WARNING] Total do Arquivo diferente do capturado no DataFrame para o Arquivo: {tt[0]}")
            print(f"[WARNING] Total catura no Dataframe: R$ {tota_df:,.2f}")
            print(f"[WARNING] Total no Arquivo: R$ {tt[1]:,.2f}")

### Vizualizando o Total Capturado

In [ ]:
df_presente.groupby('Arquivo')['Valor'].sum()

### Validando Contagem de Compras

In [ ]:
df_presente.groupby('Arquivo')['Valor'].count()